In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
batch_num = 1024
hidden_num = 150
step_num = 8
elem_num = 1
max_sequence_len = step_num
iteration = 400000

In [ ]:
tf.reset_default_graph()

In [ ]:
Y = tf.placeholder(tf.float32, shape=[None,1])
data = tf.placeholder(shape=[batch_num, max_sequence_len,elem_num],dtype=tf.float32)
#####
lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_num)
value, _ = tf.nn.dynamic_rnn(lstm_cell, data, dtype=tf.float32)
#####
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
Y_p_1 = tf.layers.dense(inputs=last,units=200,activation=tf.nn.tanh)
Y_p_2 = tf.layers.dense(inputs=Y_p_1,units=1)
cls_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=Y_p_2))
optimizer = tf.train.AdamOptimizer().minimize(cls_loss)
#####
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
loss_array = np.zeros(iteration)
for i in range(iteration):
        #step_num = np.random.random_integers(6,12)
        random_sequences = np.random.binomial(n=1,p=0.5,size=[batch_num,step_num,elem_num])
        sums = np.sum(random_sequences, axis=1)
        labels = np.mod(sums, 2)
        (loss_val, _) = sess.run([cls_loss,optimizer], {data: random_sequences,Y:labels})
        loss_array[i] = loss_val
        if ((i+1) % 1000) is 0:
            print('iter %d:' % (i + 1), loss_val)
        if loss_val < 0.001:
            loss_array = loss_array[0:i+1]
            print('LEARNED')
            break

In [ ]:
plt.plot(loss_array)
plt.show()